<a href="https://colab.research.google.com/github/pyy0715/My-Kaggle-Struggle/blob/master/nlp_tutorial(5)_XG_Boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# API Token 다운받기
# 다운받은 API Token 업로드 하기
!pip install kaggle
from google.colab import files
files.upload()
# json 파일 옮겨주기
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# json 파일 제대로 업로드 됐는지 확인
!ls -1ha kaggle.json
# 데이터셋 다운로드 받기 - 링크는 그 대회 'Data'에 있음
! kaggle competitions download -c word2vec-nlp-tutorial
# 다운로드 된 것들 다 보기 
!ls

Saving kaggle.json to kaggle.json
kaggle.json
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 76.0MB/s]
 35% 9.00M/26.0M [00:00<00:01, 13.5MB/s]
100% 26.0M/26.0M [00:00<00:00, 33.4MB/s]
 71% 9.00M/12.6M [00:00<00:00, 14.3MB/s]
100% 12.6M/12.6M [00:00<00:00, 19.9MB/s]
 69% 9.00M/13.0M [00:00<00:00, 14.9MB/s]
100% 13.0M/13.0M [00:00<00:00, 22.1MB/s]
kaggle.json		  sample_data		testData.tsv.zip
labeledTrainData.tsv.zip  sampleSubmission.csv	unlabeledTrainData.tsv.zip


In [2]:
!unzip labeledTrainData.tsv.zip
!unzip testData.tsv.zip

Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    
Archive:  testData.tsv.zip
  inflating: testData.tsv            


In [0]:
import pandas as pd
import numpy as np


# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('labeledTrainData.tsv',header=0, delimiter='\t', quoting=3)

# 레이블이 없는 테스트 데이터
test = pd.read_csv('testData.tsv', header=0, delimiter='\t', quoting=3)

In [0]:
import re
import nltk

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from multiprocessing import Pool

class KaggleWord2VecUtility(object):

    @staticmethod
    def review_to_wordlist(review, remove_stopwords=False):
        # 1. HTML 제거
        review_text = BeautifulSoup(review, "html.parser").get_text()
        # 2. 특수문자를 공백으로 바꿔줌
        review_text = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. 소문자로 변환 후 나눈다.
        words = review_text.lower().split()
        # 4. 불용어 제거
        if remove_stopwords:
            stops = set(stopwords.words('english'))
            words = [w for w in words if not w in stops]
        # 5. 어간추출
        stemmer = SnowballStemmer('english')
        words = [stemmer.stem(w) for w in words]
        # 6. 리스트 형태로 반환
        return(words)

    @staticmethod
    def review_to_join_words( review, remove_stopwords=False ):
        words = KaggleWord2VecUtility.review_to_wordlist(\
            review, remove_stopwords=False)
        join_words = ' '.join(words)
        return join_words

    @staticmethod
    def review_to_sentences( review, remove_stopwords=False ):
        # punkt tokenizer를 로드한다.
        """
        이 때, pickle을 사용하는데
        pickle을 통해 값을 저장하면 원래 변수에 연결 된 참조값 역시 저장된다.
        저장된 pickle을 다시 읽으면 변수에 연결되었던
        모든 레퍼런스가 계속 참조 상태를 유지한다.
        """
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        # 1. nltk tokenizer를 사용해서 단어로 토큰화 하고 공백 등을 제거한다.
        raw_sentences = tokenizer.tokenize(review.strip())
        # 2. 각 문장을 순회한다.
        sentences = []
        for raw_sentence in raw_sentences:
            # 비어있다면 skip
            if len(raw_sentence) > 0:
                # 태그제거, 알파벳문자가 아닌 것은 공백으로 치환, 불용어제거
                sentences.append(\
                    KaggleWord2VecUtility.review_to_wordlist(\
                    raw_sentence, remove_stopwords))
        return sentences


    # 참고 : https://gist.github.com/yong27/7869662
    # http://www.racketracer.com/2016/07/06/pandas-in-parallel/
    # 속도 개선을 위해 멀티 스레드로 작업하도록
    @staticmethod
    def _apply_df(args):
        df, func, kwargs = args
        return df.apply(func, **kwargs)

    @staticmethod
    def apply_by_multiprocessing(df, func, **kwargs):
        # 키워드 항목 중 workers 파라메터를 꺼냄
        workers = kwargs.pop('workers')
        # 위에서 가져온 workers 수로 프로세스 풀을 정의
        pool = Pool(processes=workers)
        # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
        result = pool.map(KaggleWord2VecUtility._apply_df, [(d, func, kwargs)
                for d in np.array_split(df, workers)])
        pool.close()
        # 작업 결과를 합쳐서 반환
        return pd.concat(result)

In [0]:
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    review_text = wordnet_lemmatizer.lemmatize(review_text)
    return review_text

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
# 학습데이터를 전처리 한다.
%time train['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    train['review'], review_to_words, workers=4)

CPU times: user 104 ms, sys: 123 ms, total: 227 ms
Wall time: 7.18 s


In [9]:
# 학습데이터와 동일하게 테스트 데이터에 대해서도 전처리 한다.
%time test['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    test['review'], review_to_words, workers=4)

CPU times: user 99.4 ms, sys: 119 ms, total: 219 ms
Wall time: 6.98 s


In [0]:
# X_train과 X_test에 리뷰 데이터를 담아주고 이 데이터를 TF-IDF를 통해 임베딩(벡터화)해본다. 
X_train = train['review_clean']
X_test = test['review_clean']

In [12]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 90000
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=90000, min_df=2,
                ngram_range=(1, 3), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None,
                vocabulary={'A', 'Aani', 'Aaron', 'Aaronic', 'Aaronical',
                            'Aaronite', 'Aaronitic', 'Aaru', 'Ab', 'Ababdeh',
                            'Ababua', 'Abadite', 'Abama', 'Abanic', 'Abantes',
                            'Abarambo', 'Abaris', 'Abasgi', 'Abassin', 'Abatua',
                            'Abba', 'Abbadide', 'Abbasside', 'Abbie', 'Abby',
                            'Abderian', 'Abderite', 'Abdiel', 'Abdominales',
                            'Abe', ...})

In [14]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf = False)),
])  
pipeline

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=90000, min_df=2,
                                 ngram_range=(1, 3), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None,
                                 vocabula...
                                             'Aaronical', 'Aaronite',
                                             'Aaronitic', 'Aaru', 'Ab',
                                             'Ababdeh', 'Ababua', 'Abadite',
                                             'Abama', 'Abanic', 'Abantes',
         

In [15]:
%time X_train_tfidf_vector = pipeline.fit_transform(X_train)

CPU times: user 6.76 s, sys: 33.6 ms, total: 6.8 s
Wall time: 6.81 s


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1278: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [16]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

235892


['A',
 'Aani',
 'Aaron',
 'Aaronic',
 'Aaronical',
 'Aaronite',
 'Aaronitic',
 'Aaru',
 'Ab',
 'Ababdeh']

In [39]:
%time X_test_tfidf_vector = pipeline.fit_transform(X_test)

CPU times: user 6.69 s, sys: 33.8 ms, total: 6.73 s
Wall time: 6.73 s


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1278: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [18]:
dist = np.sum(X_train_tfidf_vector, axis=0)
    
for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

[[0. 0. 0. ... 0. 0. 0.]] A


,A,Aani,Aaron,Aaronic,Aaronical,Aaronite,Aaronitic,Aaru,Ab,Ababdeh,Ababua,Abadite,Abama,Abanic,Abantes,Abarambo,Abaris,Abasgi,Abassin,Abatua,Abba,Abbadide,Abbasside,Abbie,Abby,Abderian,Abderite,Abdiel,Abdominales,Abe,Abel,Abelia,Abelian,Abelicea,Abelite,Abelmoschus,Abelonian,Abencerrages,Aberdeen,Aberdonian,...,zymic,zymin,zymite,zymogen,zymogene,zymogenesis,zymogenic,zymogenous,zymoid,zymologic,zymological,zymologist,zymology,zymolyis,zymolysis,zymolytic,zymome,zymometer,zymomin,zymophore,zymophoric,zymophosphate,zymophyte,zymoplastic,zymoscope,zymosimeter,zymosis,zymosterol,zymosthenic,zymotechnic,zymotechnical,zymotechnics,zymotechny,zymotic,zymotically,zymotize,zymotoxic,zymurgy,zythem,zythum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# XG_Boost

In [67]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_tfidf_vector, label=train['sentiment'])

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [102]:
# 멀티프로세싱은 nthread 였는데 n_jobs로 변경되었다고 한다.
# 설치 된 xgboost버전에 따라 파라메터가 다를 수 있으니 
# 이 코드를 돌리고 터미널에서 $ top -o cpu 로 CPU자원을 100%넘게 사용하고 있는지 확인해 본다.
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth' : 7,
    'n_jobs' : 4
}

%time booster = xgb.train(params, dtrain, num_boost_round=300)

CPU times: user 7min 21s, sys: 121 ms, total: 7min 22s
Wall time: 3min 43s


In [103]:
dtest = xgb.DMatrix(X_test_tfidf_vector)

result = booster.predict(dtest)

print(result.shape)
result[0:10]

(25000,)


array([9.8691911e-01, 5.1436871e-03, 8.7154359e-01, 7.0893699e-01,
       8.4170997e-01, 1.2150814e-01, 1.3488032e-02, 5.3440422e-01,
       4.1205288e-04, 4.3966568e-01], dtype=float32)

In [104]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",0.986919
1,"""8348_2""",0.005144
2,"""5828_4""",0.871544
3,"""7186_2""",0.708937
4,"""12128_7""",0.841710


In [0]:
output['sentiment']=np.where(output['sentiment']>=0.5,1,0)

output['sentiment'] = output['sentiment'].astype(int)

In [106]:
output['sentiment'].value_counts()

1    12765
0    12235
Name: sentiment, dtype: int64

In [0]:
output.to_csv("tutorial_5_xgboost.csv", index=False, quoting=3)

In [97]:
!kaggle competitions submit -c word2vec-nlp-tutorial -f tutorial_5_xgboost.csv -m 'max_depth=7,num=300'

100% 276k/276k [00:14<00:00, 19.4kB/s]
Successfully submitted to Bag of Words Meets Bags of Popcorn

In [98]:
!kaggle competitions submissions -c word2vec-nlp-tutorial

fileName                      date                 description                 status    publicScore  privateScore  
----------------------------  -------------------  --------------------------  --------  -----------  ------------  
tutorial_5_xgboost.csv        2019-06-03 07:44:36  max_depth=7                 complete  0.85540      None          
tutorial_5_xgboost.csv        2019-06-03 07:43:23  max_depth=7                 complete  0.85540      None          
tutorial_5_xgboost.csv        2019-06-03 07:36:51  num_boost_round=300         complete  0.80500      None          
tutorial_5_xgboost.csv        2019-06-03 07:26:23  num_boost_round=100         complete  0.76156      None          
tutorial_4_tfidf_0.92051.csv  2019-06-02 20:56:51  None                        complete  0.84388      None          
bag of words                  2019-05-27 09:54:19  From "bag of words" Script  complete  0.81792      None          
bag of words                  2019-05-27 09:52:35  From "bag of 